## IDS Project
#### This project will create an IDS Detection Machine Learning model, which will predict DoS attacks and flag them as such. 
By Chadi Amzil

In [ ]:
!pip install pandas
!pip install seaborn
!pip install pyarrow
!pip install xgboost
!pip install catboost
!pip install flask
!pip install catboost
!pip install hdbscan
!pip install sklearn
!pip install numpy
!pip install matplotlib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import glob
import joblib

### Extract

In [6]:
#Defining the extract function
def extract() -> pd.DataFrame:
    """This functions combines all the csv and json files into a dataframe

    Args:
        None: The function reads all csv and json files in the working directory
        
    Returns:
        data (pd.DataFrame): All data sources combined in a single dataframe
    """
    ## create an empty dataframe to hold all the data
    # The columns' names are given based on an earlier exploration of a csv file
    data = []
    ## glob.glob('*.csv') returns ['ids_0.csv', 'ids_1.csv', 'ids_2.csv']
    # 
    for csvfile in glob.glob('*.csv'):
        #adding the extracted csv files into the dataframe
        data.append(pd.read_csv(csvfile))

    
    ## glob.glob('*.json') 
    # returns ['ids_3.json', 'ids_4.json', 'ids_7.json', 'ids_9.json', 'ids_10.json]
    for jsonfile in glob.glob('*.json'):
        # adding the extracted json files into the dataframe
        data.append(pd.read_json(jsonfile, lines=True))


    ## glob.glob('*.parquet') 
    # returns ['ids_5.parquet', 'ids_6.parquet', 'ids_8.parquet', 'ids_11.parquet']
    for parquetfile in glob.glob('*.parquet'):
        # adding the extracted json files into the dataframe
        data.append(pd.read_parquet(parquetfile))
    # concatenate the loaded data to the data frame
    data = pd.concat(data, ignore_index=True)
    return data


In [7]:
# Extracting the dataframe
df = extract()
# Printing head to confirm extraction
df.head()


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,87261,1,1,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,443,523,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,53,94304,1,1,56,338,56,56,56.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,53,207,2,2,84,246,42,42,42.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,123,69022032,2,2,96,96,48,48,48.0,0.0,...,32,22276.0,0.0,22276,22276,69000000.0,0.0,69000000,69000000,BENIGN


In [ ]:
#Confriming shape of extracting dataframe
df.shape

(61128, 79)

### Transform